## Objective
**Airbnb** is an online marketplace that connects people who want to rent out their homes with people looking for accommodations in that locale. 
The Aim is to understand how Airbnb hosts can make simple changes to their properties to boost customer satisfaction.  We used a **Classification model** for Rating Prediction, and the algorithm used here is **Factorization Machines Classifier**. This experiment predicts Customer's behavior in classifying their reviews as high rated or low rated using feature Review Score Rating.

## Import Spark SQL and Spark ML Libraries
Import all the Spark SQL and ML libraries as mentioned below. This is neccessary to access the functions available in those libraries.

In [0]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import FMClassifier
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

## To run the code in PySpark CLI
Set the following to True:
```
PYSPARK_CLI = True
```
Generate .py(Python) file from Databricks: File > Export > Source File
```
Run it at the Hadoop/Spark cluster:
$ spark-submit DecisionTreeClassifier.py
```

In [0]:
PYSPARK_CLI = False
if PYSPARK_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

##Read csv file from DBFS  (Databricks File System)
1. After <filename>.csv file is added to the data in the left frame, create a table using the UI, "Upload File"
2. Click "Preview Table to view the table" and select the option as <filename>.csv has a header as the first row. "First line is header"
3. Change the data type of the columns
4. Click on the create table button.
  
 The link to the sampled file : https://www.kaggle.com/samyukthamurali/airbnb-ratings-dataset?select=airbnb_sample.csv. You can download the sample file from here and upload it in DBFS.

In [0]:
# File location and type
file_location = "/FileStore/tables/airbnb_sample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#display(df)

##Create a temporary view of the dataframe 'df'

In [0]:
# Create a view or table
temp_table_name = "airbnb_sample_csv"
df.createOrReplaceTempView(temp_table_name)

##Create a dataframe from the table, using Spark SQL

In [0]:
if PYSPARK_CLI:
    csv = spark.read.csv('airbnb_sample', inferSchema=True, header=True)
else:
    csv = spark.sql("SELECT * FROM airbnb_sample_csv")
    
csv.show(5)

+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
Listing ID|Last Scraped| Host ID| Host Name|Host Since| Host Location|Host Response Time|Host Response Rate|Host Acceptance Rate| Host Neighborhood|Host Listings Count|Host Total Listings Count| Street| Neighborhood|Neighborhood Cleansed|Neighborhood Group Cleansed| City|State|Zipcode| Market| Country| Latitude| Longitude|Property Type| Room Type|Accomodates|Bathrooms|Bedrooms|Beds|Bed Type| Amenities|Square Feet|Price|Weekly Price|Monthly Price|Security Deposit|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Maximum Nights|Calendar Updated|Has Availability|Availability 30|Availability 60|Availability 90|Availability 365|Calendar last scraped|Number of Reviews|First Review|Last Review|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Reviews per month| Geolocation|Sentiment|
+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
 4917301| 5/2/17|24035721| Michael| 11/22/14|Los Angeles, Cali...| null| null| null|Woodland Hills/Wa...| 1| 1|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null| Los Angeles| CA| 91364|Los Angeles|United States|34.14775649|-118.5913378| Apartment| Private room| 2| 1.0| 1| 1|Real Bed|TV,Wireless Inter...| null| 150| null| null| null| null| 1| 0| 1| 1125| 29 months ago| null| 30| 60| 90| 365| 5/2/17| 0| null| null| null| null| null| null| null| null| null| flexible| 1| null|34.1477564923,-11...| null|
 8384432| 5/3/17| 506809| Ky| 4/13/11|Los Angeles, Cali...|within a few hours| 100| null|Woodland Hills/Wa...| 2| 2|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null|Woodland Hills| CA| 91364|Los Angeles|United States|34.14859443|-118.5966837| Other|Entire home/apt| 2| 1.5| 1| 1|Real Bed|TV,Cable TV,Wirel...| null| 175| null| null| 500| 100| 1| 0| 2| 5| 2 weeks ago| null| 12| 42| 72| 72| 5/3/17| 2| 5/3/16| 9/20/16| 100| 10

In [0]:

csv = csv.withColumn("Review Scores Rating", when(col("Review Scores Rating") >= 80,1).otherwise(0))
csv.show(5)


+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
Listing ID|Last Scraped| Host ID| Host Name|Host Since| Host Location|Host Response Time|Host Response Rate|Host Acceptance Rate| Host Neighborhood|Host Listings Count|Host Total Listings Count| Street| Neighborhood|Neighborhood Cleansed|Neighborhood Group Cleansed| City|State|Zipcode| Market| Country| Latitude| Longitude|Property Type| Room Type|Accomodates|Bathrooms|Bedrooms|Beds|Bed Type| Amenities|Square Feet|Price|Weekly Price|Monthly Price|Security Deposit|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Maximum Nights|Calendar Updated|Has Availability|Availability 30|Availability 60|Availability 90|Availability 365|Calendar last scraped|Number of Reviews|First Review|Last Review|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Reviews per month| Geolocation|Sentiment|
+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
 4917301| 5/2/17|24035721| Michael| 11/22/14|Los Angeles, Cali...| null| null| null|Woodland Hills/Wa...| 1| 1|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null| Los Angeles| CA| 91364|Los Angeles|United States|34.14775649|-118.5913378| Apartment| Private room| 2| 1.0| 1| 1|Real Bed|TV,Wireless Inter...| null| 150| null| null| null| null| 1| 0| 1| 1125| 29 months ago| null| 30| 60| 90| 365| 5/2/17| 0| null| null| 0| null| null| null| null| null| null| flexible| 1| null|34.1477564923,-11...| null|
 8384432| 5/3/17| 506809| Ky| 4/13/11|Los Angeles, Cali...|within a few hours| 100| null|Woodland Hills/Wa...| 2| 2|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null|Woodland Hills| CA| 91364|Los Angeles|United States|34.14859443|-118.5966837| Other|Entire home/apt| 2| 1.5| 1| 1|Real Bed|TV,Cable TV,Wirel...| null| 175| null| null| 500| 100| 1| 0| 2| 5| 2 weeks ago| null| 12| 42| 72| 72| 5/3/17| 2| 5/3/16| 9/20/16| 1| 10| 10|

##Selecting features
In the following step, we are selecting the features that are useful for Rating Prediction.

In [0]:
csv = csv.filter(col("Minimum Nights")<= 365)

data = csv.select("Host Response Time","Host Response Rate","Host Acceptance Rate","Host Neighborhood","Host Listings Count","Host Total Listings Count","Property Type","Room Type","Price","Weekly Price","Monthly Price","Maximum Nights","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication","Review Scores Location","Review Scores Value","Cancellation Policy","Calculated host listings count","Neighborhood Cleansed","Neighborhood Group Cleansed","Bedrooms","Bathrooms","Beds","Security Deposit","Cleaning Fee","Extra People","Minimum Nights","Calendar Updated","Amenities", col("Review Scores Rating").alias("label"))

data.show(5)

#display(data.describe())

+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+-----+
Host Response Time|Host Response Rate|Host Acceptance Rate| Host Neighborhood|Host Listings Count|Host Total Listings Count|Property Type| Room Type|Price|Weekly Price|Monthly Price|Maximum Nights|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Neighborhood Cleansed|Neighborhood Group Cleansed|Bedrooms|Bathrooms|Beds|Security Deposit|Cleaning Fee|Extra People|Minimum Nights|Calendar Updated| Amenities|label|
+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+-----+
 null| null| null|Woodland Hills/Wa...| 1| 1| Apartment| Private room| 150| null| null| 1125| null| null| null| null| null| null| flexible| 1| Woodland Hills| null| 1| 1.0| 1| null| null| 0| 1| 29 months ago|TV,Wireless Inter...| 0|
within a few hours| 100| null|Woodland Hills/Wa...| 2| 2| Other|Entire home/apt| 175| null| null| 5| 10| 10| 10| 10| 10| 10| strict| 2| Woodland Hills| null| 1| 1.5| 1| 500| 100| 0| 2| 2 weeks ago|TV,Cable TV,Wirel...| 1|
 null| null| null| null| 1| 1| House| Private room| 75| 490| 2100| 1125| null| null| null| null| null| null| strict| 1| Woodland Hills| null| 1| 1.0| 1| 300| 60| 20| 1| 6 months ago|TV,Cable TV,Inter...| 0|
 null| null| null| West Hollywood| 1| 1| House|Entire home/apt| 146| null| null| 10| null| null| null| null| null| null| strict| 1| West Hollywood| null| 2| 1.5| 2| 500| 50| 10| 3| 17 months ago|TV,Internet,Wirel...| 0|
within a few hours| 100| null| Justicia| 3| 3| Apartment| Private room| 94| 560| 1900| 30| 10| 10| 10| 10| 10| 10| strict| 1| West Hollywood| null| 1| 1.0| 1| null| null| 0| 5| 6 weeks ago|TV,Cable TV,Inter...| 1|
+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+-----+
only showing top 5 rows

##Data Cleaning
**Handling Missing Values:** Filling the missing values of numeric columns with **'0'** and string columns with **'NA'**

In [0]:
data_clean = data.na.fill(value=0).na.fill("")
data_clean.show(5)


+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+-----+
Host Response Time|Host Response Rate|Host Acceptance Rate| Host Neighborhood|Host Listings Count|Host Total Listings Count|Property Type| Room Type|Price|Weekly Price|Monthly Price|Maximum Nights|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Neighborhood Cleansed|Neighborhood Group Cleansed|Bedrooms|Bathrooms|Beds|Security Deposit|Cleaning Fee|Extra People|Minimum Nights|Calendar Updated| Amenities|label|
+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+-----+
 | 0| |Woodland Hills/Wa...| 1| 1| Apartment| Private room| 150| 0| 0| 1125| 0| 0| 0| 0| 0| 0| flexible| 1| Woodland Hills| | 1| 1.0| 1| 0| 0| 0| 1| 29 months ago|TV,Wireless Inter...| 0|
within a few hours| 100| |Woodland Hills/Wa...| 2| 2| Other|Entire home/apt| 175| 0| 0| 5| 10| 10| 10| 10| 10| 10| strict| 2| Woodland Hills| | 1| 1.5| 1| 500| 100| 0| 2| 2 weeks ago|TV,Cable TV,Wirel...| 1|
 | 0| | | 1| 1| House| Private room| 75| 490| 2100| 1125| 0| 0| 0| 0| 0| 0| strict| 1| Woodland Hills| | 1| 1.0| 1| 300| 60| 20| 1| 6 months ago|TV,Cable TV,Inter...| 0|
 | 0| | West Hollywood| 1| 1| House|Entire home/apt| 146| 0| 0| 10| 0| 0| 0| 0| 0| 0| strict| 1| West Hollywood| | 2| 1.5| 2| 500| 50| 10| 3| 17 months ago|TV,Internet,Wirel...| 0|
within a few hours| 100| | Justicia| 3| 3| Apartment| Private room| 94| 560| 1900| 30| 10| 10| 10| 10| 10| 10| strict| 1| West Hollywood| | 1| 1.0| 1| 0| 0| 0| 5| 6 weeks ago|TV,Cable TV,Inter...| 1|
+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+-----+
only showing top 5 rows

##Convert the string type columns into indices using StringIndexer

In [0]:
data_clean = StringIndexer(inputCol='Host Response Time', outputCol='Host_Response_index').fit(data_clean).transform(data_clean)
data_clean = StringIndexer(inputCol='Room Type', outputCol='RoomType_index').fit(data_clean).transform(data_clean)
data_clean = StringIndexer(inputCol='Property Type', outputCol='PropertyType_index').fit(data_clean).transform(data_clean)
data_clean = StringIndexer(inputCol='Cancellation Policy', outputCol='Cancellation_index').fit(data_clean).transform(data_clean)

## Split the data
In the next step we split the data in a train and test set. We have split the data in the ratio of **70 to 30**.

In [0]:
# Split the data
splits = data_clean.randomSplit([0.7, 0.3])

# for Factorization Machines classifier
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Training Rows: 35842 Testing Rows: 15325

## Define the Pipeline
Define a pipeline that creates a feature vector and trains a regression model
1. A **VectorAssembler** that combines categorical features into a single vector.
2. A **Vector Indexer** that creates indices for a vector of categorical features.
3. A **VectorAssembler** that creates a vector of continuous numeric features.
4. A **MinMaxScaler** to normalize the continuous numeric features.
5. A **VectorAssembler** that creates a vector of categorical and continuous features.
6. A **Factorization Machines Classifier** that trains a Classification model.
7. **Process pipeline** with the series of transformations above.

In [0]:
catVect = VectorAssembler(inputCols = ["Host_Response_index", "RoomType_index", "PropertyType_index","Cancellation_index"], outputCol="catFeatures")

catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures").setHandleInvalid("skip") 

numVect = VectorAssembler(inputCols = ["Host Response Rate","Host Listings Count","Host Total Listings Count","Price","Weekly Price","Monthly Price","Maximum Nights","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication","Review Scores Location","Review Scores Value","Calculated host listings count","Bedrooms","Bathrooms","Beds","Security Deposit","Cleaning Fee","Extra People","Minimum Nights"], outputCol="numFeatures")

minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"],  outputCol="features")

fmc = FMClassifier(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[catVect,catIdx,numVect, minMax,featVect, fmc])


### Train a Classification model using Parameter Tuning
Use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple folds of the data split into training and validation datasets, in order to find the best performing parameters. It is used to find the best model for the data. Here the number of folds is assigned to **2**.

In [0]:
paramGrid = ParamGridBuilder() \
            .addGrid(fmc.maxIter, [5,100]) \
            .addGrid(fmc.tol, [1e-06]) \
            .addGrid(fmc.stepSize, [0.03]) \
            .addGrid(fmc.solver, ["adamW"]) \
            .addGrid(fmc.seed, [123]) \
            .build()


In [0]:
val = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=BinaryClassificationEvaluator(), numFolds=2)

In [0]:
model=val.fit(train)
#model=pipeline.fit(train)

### Test the Pipeline Model
The model produced by the pipeline is a transformer that will apply all of the stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, we will transform the **test** DataFrame using the pipeline to generate label predictions.

In [0]:

prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
#predicted = prediction.select("prediction", "trueLabel")
prediction.show()

+------------------+------------------+--------------------+-----------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+---------+-------------------+--------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
Host Response Time|Host Response Rate|Host Acceptance Rate|Host Neighborhood|Host Listings Count|Host Total Listings Count|Property Type| Room Type|Price|Weekly Price|Monthly Price|Maximum Nights|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Neighborhood Cleansed|Neighborhood Group Cleansed|Bedrooms|Bathrooms|Beds|Security Deposit|Cleaning Fee|Extra People|Minimum Nights|Calendar Updated| Amenities|trueLabel|Host_Response_index|RoomType_index|PropertyType_index|Cancellation_index| catFeatures| idxCatFeatures| numFeatures| normFeatures| features| rawPrediction| probability|prediction|
+------------------+------------------+--------------------+-----------------+-------------------+-------------------------+-------------+---------------+-----+------------+-------------+--------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+---------------------+---------------------------+--------+---------+----+----------------+------------+------------+--------------+----------------+--------------------+---------+-------------------+--------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 | 0| | | 0| 0| Apartment|Entire home/apt| 125| 700| 2000| 1125| 10| 9| 10| 10| 10| 10| moderate| 2| Upper West Side| Manhattan| 0| 1.0| 1| 0| 125| 0| 30| 2 months ago|TV,Cable TV,Inter...| 1| 1.0| 0.0| 0.0| 2.0|[1.0,0.0,0.0,2.0]|[1.0,0.0,0.0,2.0]|[0.0,0.0,0.0,125....|[0.0,0.0,0.0,0.12...|[1.0,0.0,0.0,2.0,...|[-5.1213342240678...|[0.00593264848656...| 1.0|
 | 0| | | 0| 0| Apartment| Private room| 37| 0| 0| 1125| 0| 0| 0| 0| 0| 0| flexible| 1| Bushwick| Brooklyn| 1| 0.0| 1| 0| 0| 0| 1| 22 months ago|Internet,Wireless...| 0| 1.0| 1.0| 0.0| 1.0|[1.0,1.0,0.0,1.0]|[1.0,1.0,0.0,1.0]|(21,[3,6,13,14,16...|(21,[3,6,13,14,16...|(25,[0,1,3,7,10,1...|[3.74933494599573...|[0.97700769528225...| 0.0|
 | 0| | | 0| 0| Apartment| Private room| 110| 0| 0| 15| 10| 8| 10| 10| 10| 8| flexible| 2| Greenwich Village| Manhattan| 1| 1.0| 2| 95| 30| 0| 4| 8 months ago|Wireless Internet...| 1| 1.0| 1.0| 0.0| 1.0|[1.0,1.0,0.0,1.0]|[1.0,1.0,0.0,1.0]|[0.0,0.0,0.0,110....|[0.0,0.0,0.0,0.11...|[1.0,1.0,0.0,1.0,...|[-3.3222591251779...|[0.03481541463672...| 1.0|
 | 0| | | 1| 1| Apartment|Entire home/apt| 80| 0| 0| 1125| 0| 0| 0| 0| 0| 0| flexible| 1| Long Beach| | 1| 1.0| 1| 0| 0| 0| 1| 5 months ago|TV,Internet,Wirel...| 0| 1.0| 0.0| 0.0| 1.0|[1.0,0.0,0.0,1.0]|[1.0,0.0,0.0,1.0]|(21,[1,2,3,6,13,1...|(21,[1,2,3,6,13,1...|(25,[0,3,5,6,7,10...|[3.02263081748764...|[0.95358610358330...| 0.0|
 | 0| | | 1| 1| Apartment|Entire home/apt| 87| 0| 0| 1125| 0| 0| 0| 0| 0| 0| flexible| 1| Long Beach| | 2| 2.0| 4| 0| 0| 0| 1| 7 months ago| | 0| 1.0| 0.0| 0.0| 1.0|[1.0,0.0,0.0,1.0]|[1.0,0.0,0.0,1.0]|(21,[1,2,3,6,13,1...|(21,[1,2,3,6,13,1...|(25,[0,3,5,6,7,10...|[3.44796281856615...

In [0]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()


+---------+------------------+
 metric| value|
+---------+------------------+
 TP| 10814.0|
 FP| 249.0|
 TN| 4228.0|
 FN| 34.0|
Precision| 0.977492542709934|
 Recall|0.9968657817109144|
+---------+------------------+

### Retrieve the Area Under Curve
There are several evaluation metrics for the Classification Model. Of these, the AUC is capable of distinguishing between classes. Higher the AUC, the better the model is at predicting 0s as 0s and 1s as 1s. By analogy, the Higher the AUC, the better the model distinguishes between the high and low ratings. You can use the **BinaryClassificationEvaluator** class to retrieve the AUC value.

In [0]:

evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR = ", aur)  

AUR = 0.9706240903193839